# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

In [2]:
from task import gini, entropy, gain

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
from task import DecisionTree

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
from task import RandomForestClassifier

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [5]:
from task import feature_importance
#    raise NotImplementedError()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [6]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [1.17338369e-04 8.49832084e-04 1.61981144e-01 1.59680014e-01
 3.29412099e-01 2.32260937e-03]


In [7]:
cl = ['a', 'b', 'c']
yy = np.array([cl[i] for i in y])

In [8]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, yy)
print("Accuracy:", np.mean(rfc.predict(X) == yy))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [6.98009468e-04 6.10386839e-04 1.53131189e-01 1.55185644e-01
 3.26488977e-01 1.22283437e-04]


In [9]:
rfc.trees[0].predict(rfc.trees[0].X_oob) == rfc.trees[0].y_oob

array([False, False, False,  True,  True,  True, False, False, False,
        True,  True, False, False,  True, False,  True, False,  True,
       False, False, False,  True,  True, False,  True, False,  True,
        True, False, False,  True, False,  True, False, False, False,
       False, False, False,  True, False,  True,  True,  True, False,
        True, False,  True,  True, False,  True, False,  True, False,
        True, False,  True,  True,  True,  True, False, False, False,
        True,  True, False, False,  True,  True, False,  True, False,
        True, False,  True,  True, False,  True,  True, False, False,
        True, False,  True, False,  True, False, False,  True, False,
        True, False, False, False, False, False, False,  True, False,
       False,  True,  True,  True,  True, False,  True, False,  True,
       False,  True, False, False,  True,  True, False,  True, False,
        True, False, False,  True,  True, False, False,  True,  True,
       False,  True,

### Задание 4 (3 балла)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [16]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [17]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

In [18]:
X.shape

(7924, 149)

In [19]:
from time import time

#### Возраст

In [21]:
from task import rfc_age
start = time()
rfc_age.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc_age.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_age), features, 20)):
    print(str(i+1) + ".", name)
end = time()
print("time", end-start)

Accuracy: 0.6418663303909206
Most important features:
1. ovsyanochan
2. mudakoff
3. 4ch
4. styd.pozor
5. rhymes
6. dayvinchik
7. pixel_stickers
8. tumblr_vacuum
9. reflexia_our_feelings
10. iwantyou
11. rapnewrap
12. pravdashowtop
13. xfilm
14. leprum
15. ohhluul
16. memeboizz
17. pustota_diary
18. ne.poverish
19. i_d_t
20. ne1party
time 249.47034692764282


#### Пол

In [22]:
from task import rfc_gender

rfc_gender = RandomForestClassifier(n_estimators=10)
rfc_gender.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_gender), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8537200504413619
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. zerofat
5. mudakoff
6. igm
7. 9o_6o_9o
8. rapnewrap
9. femalemem
10. i_d_t
11. cook_good
12. thesmolny
13. be.women
14. bot_maxim
15. be.beauty
16. reflexia_our_feelings
17. beauty
18. combovine
19. bon
20. 4ch


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [27]:
X, y = synthetic_dataset(1000)
clf.fit(X, y)
y_pred = clf.predict(X)
print("Accuracy:", np.mean(y_pred == y))
print("Importance:", clf.feature_importances_)

0:	learn: 0.9209051	total: 824us	remaining: 81.6ms
1:	learn: 0.7818043	total: 1.71ms	remaining: 83.9ms
2:	learn: 0.6756673	total: 2.29ms	remaining: 74.2ms
3:	learn: 0.5891762	total: 2.93ms	remaining: 70.3ms
4:	learn: 0.5151783	total: 3.54ms	remaining: 67.4ms
5:	learn: 0.4530725	total: 4ms	remaining: 62.6ms
6:	learn: 0.4003092	total: 4.44ms	remaining: 59ms
7:	learn: 0.3564962	total: 5ms	remaining: 57.5ms
8:	learn: 0.3185006	total: 5.58ms	remaining: 56.5ms
9:	learn: 0.2841434	total: 6.1ms	remaining: 54.9ms
10:	learn: 0.2552337	total: 6.68ms	remaining: 54.1ms
11:	learn: 0.2287282	total: 7.16ms	remaining: 52.5ms
12:	learn: 0.2063578	total: 7.71ms	remaining: 51.6ms
13:	learn: 0.1855926	total: 8.19ms	remaining: 50.3ms
14:	learn: 0.1680711	total: 8.73ms	remaining: 49.5ms
15:	learn: 0.1524595	total: 9.29ms	remaining: 48.8ms
16:	learn: 0.1418847	total: 10.2ms	remaining: 50ms
17:	learn: 0.1290646	total: 10.8ms	remaining: 49.2ms
18:	learn: 0.1168278	total: 11.3ms	remaining: 48.1ms
19:	learn: 0.10

In [ ]:
clf.feature_importances_

### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [91]:
clf = CatBoostClassifier(n_estimators=200, min_data_in_leaf=20, max_depth=10, loss_function='MultiClass')

In [101]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [80]:
np.mean(y_pred.flatten() == y_age_test)

0.7477931904161412

In [93]:
clf.fit(X_train, y_age_train)
y_pred = clf.predict(X_test).flatten()
print("Accuracy:", np.mean(y_pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Learning rate set to 0.336913
0:	learn: 0.9818203	total: 138ms	remaining: 27.4s
1:	learn: 0.9088327	total: 283ms	remaining: 28.1s
2:	learn: 0.8597561	total: 435ms	remaining: 28.6s
3:	learn: 0.8152467	total: 593ms	remaining: 29.1s
4:	learn: 0.7817335	total: 751ms	remaining: 29.3s
5:	learn: 0.7590963	total: 918ms	remaining: 29.7s
6:	learn: 0.7375611	total: 1.07s	remaining: 29.6s
7:	learn: 0.7153442	total: 1.22s	remaining: 29.4s
8:	learn: 0.6981209	total: 1.38s	remaining: 29.3s
9:	learn: 0.6809334	total: 1.54s	remaining: 29.2s
10:	learn: 0.6666808	total: 1.68s	remaining: 28.9s
11:	learn: 0.6540741	total: 1.83s	remaining: 28.7s
12:	learn: 0.6425774	total: 1.98s	remaining: 28.5s
13:	learn: 0.6316478	total: 2.12s	remaining: 28.2s
14:	learn: 0.6220928	total: 2.26s	remaining: 27.9s
15:	learn: 0.6137078	total: 2.41s	remaining: 27.7s
16:	learn: 0.6057425	total: 2.57s	remaining: 27.7s
17:	learn: 0.5977851	total: 2.72s	remaining: 27.5s
18:	learn: 0.5902669	total: 2.87s	remaining: 27.4s
19:	learn: 

161:	learn: 0.2245631	total: 23.4s	remaining: 5.49s
162:	learn: 0.2230793	total: 23.5s	remaining: 5.34s
163:	learn: 0.2220582	total: 23.7s	remaining: 5.2s
164:	learn: 0.2210644	total: 23.8s	remaining: 5.05s
165:	learn: 0.2200600	total: 24s	remaining: 4.91s
166:	learn: 0.2190791	total: 24.1s	remaining: 4.76s
167:	learn: 0.2180779	total: 24.2s	remaining: 4.62s
168:	learn: 0.2169274	total: 24.4s	remaining: 4.47s
169:	learn: 0.2158644	total: 24.5s	remaining: 4.33s
170:	learn: 0.2148892	total: 24.7s	remaining: 4.18s
171:	learn: 0.2142747	total: 24.8s	remaining: 4.04s
172:	learn: 0.2134825	total: 24.9s	remaining: 3.89s
173:	learn: 0.2127460	total: 25.1s	remaining: 3.75s
174:	learn: 0.2118997	total: 25.2s	remaining: 3.6s
175:	learn: 0.2109294	total: 25.4s	remaining: 3.46s
176:	learn: 0.2099285	total: 25.6s	remaining: 3.32s
177:	learn: 0.2090588	total: 25.7s	remaining: 3.18s
178:	learn: 0.2079300	total: 25.9s	remaining: 3.03s
179:	learn: 0.2072312	total: 26s	remaining: 2.89s
180:	learn: 0.2063

In [82]:
clf.save_model("model_age")

In [83]:
y_pred = xgb.predict(X_test).flatten()
print("Accuracy:", np.mean(y_pred == y_age_test))

Accuracy: 0.7477931904161412


#### Пол

In [102]:
clf1 = CatBoostClassifier(n_estimators=200, min_data_in_leaf=30, max_depth=7, loss_function='MultiClass')

In [103]:
clf1.fit(X_train, y_sex_train)
y_pred = clf1.predict(X_test).flatten()
print("Accuracy:", np.mean(y_pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf1.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Learning rate set to 0.336913
0:	learn: 0.6188799	total: 11.4ms	remaining: 2.27s
1:	learn: 0.5714152	total: 23.6ms	remaining: 2.34s
2:	learn: 0.5370335	total: 36ms	remaining: 2.36s
3:	learn: 0.5065943	total: 49.9ms	remaining: 2.45s
4:	learn: 0.4861830	total: 63.6ms	remaining: 2.48s
5:	learn: 0.4706003	total: 75.6ms	remaining: 2.44s
6:	learn: 0.4540057	total: 87.5ms	remaining: 2.41s
7:	learn: 0.4386258	total: 99.4ms	remaining: 2.38s
8:	learn: 0.4260635	total: 112ms	remaining: 2.38s
9:	learn: 0.4167468	total: 125ms	remaining: 2.38s
10:	learn: 0.4066223	total: 137ms	remaining: 2.36s
11:	learn: 0.3995241	total: 149ms	remaining: 2.33s
12:	learn: 0.3902590	total: 162ms	remaining: 2.33s
13:	learn: 0.3842166	total: 174ms	remaining: 2.31s
14:	learn: 0.3784623	total: 186ms	remaining: 2.29s
15:	learn: 0.3727293	total: 213ms	remaining: 2.45s
16:	learn: 0.3690681	total: 225ms	remaining: 2.43s
17:	learn: 0.3647909	total: 237ms	remaining: 2.4s
18:	learn: 0.3609908	total: 248ms	remaining: 2.37s
19:	le

160:	learn: 0.1902333	total: 2.06s	remaining: 498ms
161:	learn: 0.1896788	total: 2.07s	remaining: 485ms
162:	learn: 0.1891359	total: 2.08s	remaining: 472ms
163:	learn: 0.1885947	total: 2.09s	remaining: 460ms
164:	learn: 0.1879525	total: 2.11s	remaining: 447ms
165:	learn: 0.1874780	total: 2.12s	remaining: 434ms
166:	learn: 0.1869372	total: 2.13s	remaining: 421ms
167:	learn: 0.1864122	total: 2.14s	remaining: 408ms
168:	learn: 0.1857789	total: 2.15s	remaining: 395ms
169:	learn: 0.1853366	total: 2.17s	remaining: 383ms
170:	learn: 0.1846805	total: 2.18s	remaining: 370ms
171:	learn: 0.1842780	total: 2.19s	remaining: 357ms
172:	learn: 0.1838069	total: 2.21s	remaining: 344ms
173:	learn: 0.1833261	total: 2.22s	remaining: 331ms
174:	learn: 0.1826601	total: 2.23s	remaining: 319ms
175:	learn: 0.1822172	total: 2.24s	remaining: 306ms
176:	learn: 0.1816212	total: 2.25s	remaining: 293ms
177:	learn: 0.1812408	total: 2.27s	remaining: 280ms
178:	learn: 0.1807972	total: 2.28s	remaining: 267ms
179:	learn: 

In [106]:
clf1.save_model("model_sex") 

In [7]:
8/12

0.6666666666666666